In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [16]:
from public_data.models import Region
from django.contrib.gis.db.models.functions import Centroid
from django.contrib.gis.db.models import Union, Extent

qs = Region.objects.aggregate(
        mpoly_union=Union("mpoly"),
        center=Centroid(Union("mpoly")),
        extent=Extent("mpoly")
)

qs

{'mpoly_union': <MultiPolygon object at 0x7f922ed75b10>,
 'center': <Point object at 0x7f922ed75810>,
 'extent': (-5.1413332610739,
  41.3336317564725,
  9.55996004666832,
  51.0890003105061)}

In [19]:
center = (qs["center"].y, qs["center"].x)
center

(46.55800750585176, 2.5498533551600318)

In [42]:
coords = (qs['extent'][1], qs['extent'][0], qs['extent'][3], qs['extent'][2])
coords

(41.3336317564725, -5.1413332610739, 51.0890003105061, 9.55996004666832)

In [25]:
x = (qs['extent'][0] + qs['extent'][2]) / 2
y = (qs['extent'][1] + qs['extent'][3]) / 2
center2 = (y, x)
center2

(46.2113160334893, 2.2093133927972097)

In [37]:
from ipyleaflet import Map, Marker, GeoJSON, Polygon as iPolygon, Icon, WKTLayer
from public_data.api.serializers import RegionSerializer
from public_data.models import Region
from django.contrib.gis.geos import Polygon as dPolygon


m = Map(center=center, zoom=5)

icon = Icon(icon_url='https://leafletjs.com/examples/custom-icons/leaf-green.png')
m.add_layer(Marker(location=center, draggable=True, icon=icon))

m.add_layer(
    iPolygon(
        locations=dPolygon.from_bbox(coords).coords,
        fillColor="red",
        fillOpacity=0.1,
    )
)

m.add_layer(Marker(location=center2, draggable=True))

m

Map(center=[46.55800750585176, 2.5498533551600318], controls=(ZoomControl(options=['position', 'zoom_in_text',…

In [17]:
m = Map(center=center, zoom=5)
serializer = RegionSerializer(Region.objects.all(), many=True)
m.add_layer(
    GeoJSON(
        data=serializer.data,
        style={
            'opacity': 1,
            'fillOpacity': 0.1,
            'weight': 1,
            'color': 'white',
            'fillColor': 'blue',
        },
    )
)
m

Map(center=[45.51461100292581, 4.536186161788389], controls=(ZoomControl(options=['position', 'zoom_in_text', …

In [43]:
m = Map(center=center, zoom=5)

m.add_layer(
    WKTLayer(
        wkt_string=dPolygon.from_bbox(coords).ewkt,
    )
)
m

Map(center=[46.55800750585176, 2.5498533551600318], controls=(ZoomControl(options=['position', 'zoom_in_text',…